# '1997-2005' Model

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import data
races = pd.read_csv('races.csv')
runs = pd.read_csv('runs.csv')
weather = pd.read_csv('weather.csv')

## Data

### Pre-process Data

In [2]:
# process run data
runs_data = runs[['race_id', 'horse_id','trainer_id','jockey_id', 'won', 'result', 'lengths_behind', 'horse_age', 'horse_country', 'horse_type', 'horse_rating', 'declared_weight', 'actual_weight', 'draw', 'win_odds', 'place_odds', 'finish_time']] # 'horse_gear'
#temp = runs.T[~runs.columns.isin(runs_data.columns)]
#print(temp.T.columns)
#temp.T.head()
runs_data.head()

,race_id,horse_id,trainer_id,jockey_id,won,result,lengths_behind,horse_age,horse_country,horse_type,horse_rating,declared_weight,actual_weight,draw,win_odds,place_odds,finish_time
0,0,3917,118,2,0.0,10,8.00,3,AUS,Gelding,60,1020.0,133,7,9.7,3.7,83.92
1,0,2157,164,57,0.0,8,5.75,3,NZ,Gelding,60,980.0,133,12,16.0,4.9,83.56
2,0,858,137,18,0.0,7,4.75,3,NZ,Gelding,60,1082.0,132,8,3.5,1.5,83.40
3,0,1853,80,59,0.0,9,6.25,3,SAF,Gelding,60,1118.0,127,13,39.0,11.0,83.62
4,0,2796,9,154,0.0,6,3.75,3,GB,Gelding,60,972.0,131,14,50.0,14.0,83.24


In [3]:
# process race data
races_data = races[['date','race_id', 'venue', 'config', 'surface', 'distance', 'going', 'race_class']]
#temp = races.T[~races.columns.isin(races_data.columns)]
#print(temp.T.columns)
#temp.T.head()
races_data.head()

,date,race_id,venue,config,surface,distance,going,race_class
0,1997-06-02,0,ST,A,0,1400,GOOD TO FIRM,5
1,1997-06-02,1,ST,A,0,1200,GOOD TO FIRM,5
2,1997-06-02,2,ST,A,0,1400,GOOD TO FIRM,4
3,1997-06-02,3,ST,A,0,1200,GOOD TO FIRM,1
4,1997-06-02,4,ST,A,0,1600,GOOD TO FIRM,4


### Merge DataFrames

In [4]:
# merge the race and run datasets based on race_id column
merge = pd.merge(runs_data, races_data, on='race_id')
print('Size b/f Nan Drop:',merge.shape)
merge = merge.dropna().reset_index(drop=True) # 79447 entries -> 75710 entries

# convert date into datetime type
merge['date'] = pd.to_datetime(merge['date']) # date if one-hotted
merge['date'] = merge['date'] + pd.to_timedelta('4123 days')

# merge weather data
weather_data = weather[['Date','Value']].rename(columns={'Date':'date','Value':'temprature'})
weather_data['date'] = pd.to_datetime(weather_data['date']) 
merge = pd.merge(merge, weather_data, on='date')

# output quick stats & merged df
avg_race_count = round(merge.horse_id.shape[0]/merge.horse_id.unique().shape[0],3)
print('Final Merged Size:',merge.shape)
print('Avg Races per Horse:',avg_race_count)
merge.head()

Size b/f Nan Drop: (79447, 24)
Final Merged Size: (75710, 25)
Avg Races per Horse: 17.689


,race_id,horse_id,trainer_id,jockey_id,won,result,lengths_behind,horse_age,horse_country,horse_type,...,place_odds,finish_time,date,venue,config,surface,distance,going,race_class,temprature
0,0,3917,118,2,0.0,10,8.00,3,AUS,Gelding,...,3.7,83.92,2008-09-15,ST,A,0,1400,GOOD TO FIRM,5,30.8
1,0,2157,164,57,0.0,8,5.75,3,NZ,Gelding,...,4.9,83.56,2008-09-15,ST,A,0,1400,GOOD TO FIRM,5,30.8
2,0,858,137,18,0.0,7,4.75,3,NZ,Gelding,...,1.5,83.40,2008-09-15,ST,A,0,1400,GOOD TO FIRM,5,30.8
3,0,1853,80,59,0.0,9,6.25,3,SAF,Gelding,...,11.0,83.62,2008-09-15,ST,A,0,1400,GOOD TO FIRM,5,30.8
4,0,2796,9,154,0.0,6,3.75,3,GB,Gelding,...,14.0,83.24,2008-09-15,ST,A,0,1400,GOOD TO FIRM,5,30.8


### Feature Extraction

In [5]:
# define average calculation function
def get_average(col):
    values = []
    averages = [np.nan]
    for row in col[1:]:
        values += [row]
        averages += [np.mean(values)]
    return averages

#horses = merge['horse_id'].unique()
#temp = merge[merge['horse_id'] == horses[0]]
#temp[['lengths_behind']].apply(get_average)

In [6]:
# all but preferences takes 79 sec
# extract horse features
horses = merge['horse_id'].unique() # isolate each unique horse

# set parameters
end = -1

# define feature Series
race_count = pd.Series(dtype='float64')
days_since_race = pd.Series(dtype='float64')

# define recent & average Series
recent_result = pd.Series(dtype='float64')
average_result = pd.Series(dtype='float64')
recent_lengths_behind = pd.Series(dtype='float64')
average_lengths_behind = pd.Series(dtype='float64')
recent_normal_speed = pd.Series(dtype='float64')
average_normal_speed = pd.Series(dtype='float64')
recent_race_class = pd.Series(dtype='float64')
average_race_class = pd.Series(dtype='float64')
recent_draw = pd.Series(dtype='float64')
average_draw = pd.Series(dtype='float64')
recent_actual_weight = pd.Series(dtype='float64')
average_actual_weight = pd.Series(dtype='float64')

# perform for each horse
for horse in horses[0:end]:
    
    # isolate horses races
    temp = merge[merge['horse_id'] == horse]
    
    # extract # of race ran
    temp['race_count'] = list(range(0,temp.shape[0])) # attach to index
    race_count = race_count.append(temp['race_count']) # append list
    
    # extract days since last race
    temp['days_since_race'] = (temp['date'] - temp['date'].shift(1))
    temp['days_since_race'] = temp['days_since_race'].map(lambda x: x.days)
    days_since_race = days_since_race.append(temp['days_since_race']) 

    # extract most recent race result
    temp['recent_result'] = temp['result'].shift(1)
    recent_result = recent_result.append(temp['recent_result']) # append list
    
    # extract average race result
    temp['average_result'] = temp[['recent_result']].apply(get_average)
    average_result = average_result.append(temp['average_result'])
    
    # extract most recent races lengths behind
    temp['recent_lengths_behind'] = temp['lengths_behind'].shift(1)
    recent_lengths_behind = recent_lengths_behind.append(temp['recent_lengths_behind']) 
    
    # extract average lengths behind (maybe try 3/5-race average later)
    temp['average_lengths_behind'] = temp[['recent_lengths_behind']].apply(get_average)
    average_lengths_behind = average_lengths_behind.append(temp['average_lengths_behind']) 
    
    # extract normalized finishing speed
    temp['recent_normal_speed'] = (temp['finish_time']/temp['distance']).shift(1)
    recent_normal_speed = recent_normal_speed.append(temp['recent_normal_speed']) 
    
    # extract average normalized finishing speed
    temp['average_normal_speed'] = temp[['recent_normal_speed']].apply(get_average)
    average_normal_speed = average_normal_speed.append(temp['average_normal_speed']) 
    
    # extract recent competition strength
    temp['recent_race_class'] = temp['race_class'].shift(1)
    recent_race_class = recent_race_class.append(temp['recent_race_class'])
    
    # extract average past competition strength
    temp['average_race_class'] = temp[['recent_race_class']].apply(get_average)
    average_race_class = average_race_class.append(temp['average_race_class'])
    
    # extract recent competition strength
    temp['recent_draw'] = temp['draw'].shift(1)
    recent_draw = recent_draw.append(temp['recent_draw'])
    
    # extract average past competition strength
    temp['average_draw'] = temp[['recent_draw']].apply(get_average)
    average_draw = average_draw.append(temp['average_draw'])
    
    # extract recent actual weight
    temp['recent_actual_weight'] = temp['actual_weight'].shift(1)
    recent_actual_weight = recent_actual_weight.append(temp['recent_actual_weight'])
    
    # extract average actual weight
    temp['average_actual_weight'] = temp[['recent_actual_weight']].apply(get_average)
    average_actual_weight = average_actual_weight.append(temp['average_actual_weight']) 
    

# define new df w/ extracted features
extracted_df = merge.copy()

# load extracted features
extracted_df['race_count'] = race_count
extracted_df['days_since_race'] = days_since_race

# load recent & average features
extracted_df['recent_result'] = recent_result
extracted_df['average_result'] = average_result
extracted_df['recent_lengths_behind'] = recent_lengths_behind
extracted_df['average_lengths_behind'] = average_lengths_behind
extracted_df['recent_normal_speed'] = recent_normal_speed
extracted_df['average_normal_speed'] = average_normal_speed
extracted_df['recent_race_class'] = recent_race_class
extracted_df['average_race_class'] = average_race_class
extracted_df['recent_draw'] = recent_draw
extracted_df['average_draw'] = average_draw
extracted_df['recent_actual_weight'] = recent_actual_weight
extracted_df['average_actual_weight'] = average_actual_weight

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [7]:
def get_preference(df,col_name,check=True):
    new_col_name = col_name + '_pref'
    df = df.reset_index(drop=True)
    preference = [np.nan]
    for i in range(1,df.shape[0]):
        
        # get col preferences
        temp_df = df.loc[:i-1,['result',col_name]]
        temp_group = temp_df.groupby(col_name)[['result']].mean().rename(columns={'result':new_col_name})
        temp = temp_df.join(temp_group, on=col_name)
        
        # check for most similar pairing
        current = df.loc[i,col_name]
        if current != temp.loc[i-1,col_name] and check == True:
            less = np.abs(current - temp[col_name])
            pref = temp.loc[less.idxmin,[new_col_name]]
        else:
            pref = temp.loc[i-1,[new_col_name]]
        preference += [pref.values[0]]
    return preference

#horses = merge['horse_id'].unique()
#temp = merge[merge['horse_id'] == horses[0]]
#temp['distance_pref'] = get_preference(temp,'distance',True)
#temp

In [8]:
# extract horse features
horses = merge['horse_id'].unique() # isolate each unique horse

# define preference Series
distance_pref = pd.Series(dtype='float64')
surface_pref = pd.Series(dtype='float64')
going_pref = pd.Series(dtype='float64')
venue_pref = pd.Series(dtype='float64')

# perform for each horse
for horse in horses[0:end]:
    
    # isolate horses races
    temp = merge[merge['horse_id'] == horse]
    
    # may need to normalize results for preference features
    # extract distance preferences (avg finish result at given distance)
    temp['distance_pref'] = get_preference(temp,'distance')
    distance_pref = distance_pref.append(temp['distance_pref'])

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# perform for each horse
for horse in horses[0:end]:
    
    # extract surface preferences (avg finish result on given surface)
    temp = merge[merge['horse_id'] == horse]    
    temp['surface_pref'] = get_preference(temp,'surface')
    surface_pref = surface_pref.append(temp['surface_pref'])

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# perform for each horse
for horse in horses[0:end]:
    
    # extract going preferences (avg finish result on given going)
    temp = merge[merge['horse_id'] == horse]    
    temp['going_pref'] = get_preference(temp,'going',False)
    going_pref = going_pref.append(temp['going_pref']) 

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# perform for each horse
for horse in horses[0:end]:
    
    # extract track preferences (avg finish result on given going)
    temp = merge[merge['horse_id'] == horse]    
    temp['venue_pref'] = get_preference(temp,'venue',False)
    venue_pref = venue_pref.append(temp['venue_pref'])

# load preference features
extracted_df['distance_pref'] = distance_pref
extracted_df['surface_pref'] = surface_pref
extracted_df['going_pref'] = going_pref
extracted_df['venue_pref'] = venue_pref

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# extract jockey features
jockies = merge['jockey_id'].unique() # isolate each unique horse

# define jockey Series
recent_jockey_prof = pd.Series(dtype='float64')
average_jockey_prof = pd.Series(dtype='float64')

# perform for each jockey
for jockey in jockies[0:end]:
    
    # isolate jockey races
    temp = merge[merge['jockey_id'] == jockey]
    
    # extract recent jocky performance
    temp['recent_jockey_prof'] = temp['result'].shift(1)
    recent_jockey_prof = recent_jockey_prof.append(temp['recent_jockey_prof'])
    
    # extract average jocky performance
    temp['average_jockey_prof'] = temp[['recent_jockey_prof']].apply(get_average)
    average_jockey_prof = average_jockey_prof.append(temp['average_jockey_prof']) 
    
# load preference features
extracted_df['recent_jockey_prof'] = recent_jockey_prof
extracted_df['average_jockey_prof'] = average_jockey_prof

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# extract trainer features
trainers = merge['trainer_id'].unique() # isolate each unique horse

# define trainer Series
recent_trainer_perf = pd.Series(dtype='float64')
average_trainer_perf = pd.Series(dtype='float64')

# perform for each trainer
for trainer in trainers[0:end]:
    
    # isolate trainer races
    temp = merge[merge['trainer_id'] == trainer]
    
    # extract recent trainer performance
    temp['recent_trainer_perf'] = temp['result'].shift(1)
    recent_trainer_perf = recent_trainer_perf.append(temp['recent_trainer_perf'])
    
    # extract average trainer performance
    temp['average_trainer_perf'] = temp[['recent_trainer_perf']].apply(get_average)
    average_trainer_perf = average_trainer_perf.append(temp['average_trainer_perf']) 

# load preference features
extracted_df['recent_trainer_perf'] = recent_trainer_perf
extracted_df['average_trainer_perf'] = average_trainer_perf

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
# extract trainer & jockey horse features
horses = merge['horse_id'].unique() # isolate each unique horse

# define horse's past jocky performance Series
recent_jockey_skill = pd.Series(dtype='float64')
average_jockey_skill = pd.Series(dtype='float64')

# perform for each horse
for horse in horses[0:end]:
    
    # isolate horses races
    temp = extracted_df[extracted_df['horse_id'] == horse]

    # extract horse's most recent jokey skill
    temp['recent_jockey_skill'] = temp['average_jockey_prof'].shift(1)
    recent_jockey_skill = recent_jockey_skill.append(temp['recent_jockey_skill']) 
    
    # extract horse's average jockey skill
    temp['average_jockey_skill'] = temp[['average_jockey_prof']].apply(get_average)
    average_jockey_skill = average_jockey_skill.append(temp['average_jockey_skill']) 

# load recent & average features
extracted_df['recent_jockey_skill'] = recent_jockey_skill
extracted_df['average_jockey_skill'] = average_jockey_skill

C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\shanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
# output extracted feature dataframe
#extracted_df.to_csv('extracted.csv',index=False)
extracted_df

,race_id,horse_id,trainer_id,jockey_id,won,result,lengths_behind,horse_age,horse_country,horse_type,...,distance_pref,surface_pref,going_pref,venue_pref,recent_jockey_prof,average_jockey_prof,recent_trainer_perf,average_trainer_perf,recent_jockey_skill,average_jockey_skill
0,0,3917,118,2,0.0,10,8.00,3,AUS,Gelding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2157,164,57,0.0,8,5.75,3,NZ,Gelding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,858,137,18,0.0,7,4.75,3,NZ,Gelding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1853,80,59,0.0,9,6.25,3,SAF,Gelding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2796,9,154,0.0,6,3.75,3,GB,Gelding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75705,6128,1153,118,40,0.0,11,8.25,4,NZ,Gelding,...,3.5000,4.250000,3.428571,4.250000,6.0,7.146191,10.0,5.614963,6.542586,5.814738
75706,6128,1925,73,161,0.0,6,5.50,6,GB,Gelding,...,6.6875,6.939394,7.727273,7.666667,2.0,7.639623,13.0,6.813481,6.632290,6.691063
75707,6128,2531,138,10,0.0,4,4.50,3,AUS,Gelding,...,4.5000,4.500000,2.500000,4.500000,9.0,6.049844,8.0,7.028095,5.948980,6.285342
75708,6128,212,7,23,0.0,9,7.00,4,AUS,Horse,...,7.5000,7.928571,8.285714,7.846154,3.0,8.291667,3.0,5.772901,6.634680,6.496382


### Feature Processing

In [10]:
# load dataframe
extracted_df = pd.read_csv('extracted.csv')

# drop columns no longer needed
extracted_df = extracted_df.drop(columns=['horse_id','jockey_id','trainer_id','finish_time','lengths_behind','date'])

# one-hot encode categorical features
extracted_df = pd.get_dummies(extracted_df, drop_first=True)

# drop each horses first race
print('Size b/f Nan Drop:',extracted_df.shape)
extracted_df = extracted_df.dropna().reset_index(drop=True)
print('Final DataFrame Size:',extracted_df.shape)

# save one-hotted dataset
extracted_df.to_csv('extracted_big.csv',index=False)

Size b/f Nan Drop: (75710, 75)
Final DataFrame Size: (70064, 75)


### Exploration

In [ ]:
def plot_relations(col1,col2):
    # grab relation data
    total_count = runs_data.groupby(col1)[col2].count()
    conditional_prob = runs_data[runs_data[col2] == 1].groupby(col1)[col2].count()/total_count
    
    # plot relations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (18,4))

    ax1.bar(total_count.index, total_count.values)
    title = 'Horses Per ' + col1
    ax1.set_title(title)
    ax1.set_ylabel('Count')
    ax1.set_xlabel(col1)

    ax2.bar(conditional_prob.index, conditional_prob.values)
    ax2.set_title('Conditional Rate')
    ylabel = 'P( ' + col2 + ' | ' + col1 + ' )'
    ax2.set_ylabel(ylabel)
    ax2.set_xlabel(col1)

    plt.show()

In [ ]:
'''plot_relations('horse_country','won')
plot_relations('horse_age','won')
plot_relations('actual_weight','won')
plot_relations('declared_weight','won')
plot_relations('draw','won')
plot_relations('horse_type','won')
plot_relations('win_odds','won')
plot_relations('place_odds','won')'''
plot = False